In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import pandas as pd
import numpy as np

In [39]:
mode = 'dpll_preprocess'

In [40]:
feature = pd.read_csv(f"/content/drive/MyDrive/cs257_23fall/{mode}/features_{mode}.csv")
# print(feature)

In [49]:
output = pd.read_csv("/content/drive/MyDrive/cs257_23fall/output_300.csv", index_col=0)
# print(output)

In [50]:
result = pd.merge(feature, output, left_on='file_name', right_on='hash', how='inner')
result = result.drop(columns=['file_name', 'satzilla_base_t', 'satzilla_probe_t', 'hash', 'benchmark'])
# result = result.drop(columns=['unit_props_at_depth_1', 'unit_props_at_depth_4', 'unit_props_at_depth_16', 'unit_props_at_depth_64',
#      'unit_props_at_depth_256', 'mean_depth_to_contradiction_over_vars', 'estimate_log_number_nodes_over_vars',
#      'saps_BestSolution_Mean', 'saps_BestSolution_CoeffVariance', 'saps_FirstLocalMinStep_Mean',
#      'saps_FirstLocalMinStep_CoeffVariance', 'saps_FirstLocalMinStep_Median', 'saps_FirstLocalMinStep_Q.10',
#      'saps_FirstLocalMinStep_Q.90', 'saps_BestAvgImprovement_Mean', 'saps_BestAvgImprovement_CoeffVariance',
#      'saps_FirstLocalMinRatio_Mean', 'saps_FirstLocalMinRatio_CoeffVariance', 'saps_EstACL_Mean',
#      'gsat_BestSolution_Mean', 'gsat_BestSolution_CoeffVariance', 'gsat_FirstLocalMinStep_Mean',
#      'gsat_FirstLocalMinStep_CoeffVariance', 'gsat_FirstLocalMinStep_Median', 'gsat_FirstLocalMinStep_Q.10',
#      'gsat_FirstLocalMinStep_Q.90', 'gsat_BestAvgImprovement_Mean', 'gsat_BestAvgImprovement_CoeffVariance',
#      'gsat_FirstLocalMinRatio_Mean', 'gsat_FirstLocalMinRatio_CoeffVariance', 'gsat_EstACL_Mean',])
# print(result)

In [51]:
options = np.array(["z3+''", "z3+'sat.local_search=true'", "z3+'sat.lookahead_simplify=true'", "z3+'sat.ddfw_search=true'"])

In [52]:
result = result[(result[f"result({options[0]})"] != 'unknown') |
                (result[f"result({options[1]})"] != 'unknown') |
                (result[f"result({options[2]})"] != 'unknown') |
                (result[f"result({options[3]})"] != 'unknown')]
# print(result)

In [53]:
def choose(row):
  t = row[[f"time({option})" for option in options]]
  return options[np.argmin(t)]

result['select'] = result.apply(choose, axis=1)
# result.to_csv(f'/content/drive/MyDrive/cs257_23fall/{mode}/result_with_time.csv', index=False)
# print(result)

In [54]:
result = result.drop(columns=[f"result({option})" for option in options] + [f"time({option})" for option in options])
# result.to_csv(f'/content/drive/MyDrive/cs257_23fall/{mode}/result.csv', index=False)

In [55]:
X = result.iloc[:,:-1]
y = result.iloc[:,-1]
X_normalized = (X-X.mean())/X.std()
result = pd.concat([X_normalized, y], axis=1)
# print(result)

In [56]:
# remove nan columns
result = result.drop(columns=['pnc_ratio_max', 'gsat_EstACL_Mean', 'saps_EstACL_Mean'])
# result = result.drop(columns=['pnc_ratio_max'])
result.to_csv(f'/content/drive/MyDrive/cs257_23fall/{mode}/result_normalized.csv', index=False)